In [2]:
import sys
# 直接导入Smote是不行的，必须将路径加入才可以
sys.path.append("C:Users\perma\PycharmProjects\paper")
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler 
from kmeans_smote import KMeansSMOTE

In [3]:
# 读取数据
column_names = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "class"]
data_path = "C:\\Users\\perma\\PycharmProjects\\paper\\data\\glass.data"
table = pd.read_table(filepath_or_buffer=data_path,
                      sep=",",
                      header=None,
                      index_col=None,
                      names=column_names)
attributes = ["RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe"]
data = table[attributes]
label = table["class"]
print("length: data, label", len(data), len(label))
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(X=data)
scaled_data[0:3]
minor_class = [5, 6, 7]
major_class = [1, 2, 3, 4]
new_label = []
for l in label:
    if l in major_class:
        new_label.append(0.0)
    else:
        new_label.append(1.0)


length: data, label 214 214


D:\software\anaconda\exe\envs\forpaper\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, make_scorer, f1_score
from sklearn.model_selection import train_test_split

In [5]:
knn = KNeighborsClassifier()
dc = DecisionTreeClassifier()
svm = SVC()
lg = LogisticRegression()
nb = GaussianNB()
algorithm = {"knn": knn, "tree": dc, "svm": svm, "lgs": lg, "nb": nb}
alg_params = {"knn": {"n_neighbors": 7}, 
              "tree": {}, 
              "svm": {}, 
              "lgs": {}, 
              "nb": {}}


In [6]:
# 这种实验有很多预测对的少数类样本是属于生成的样本，并不能代表对实际少数类的真实预测情况
# original的可以代表对真实的预测情况
# 所以决定用合成少数类来训练样本，把真实的样本留在预测
# 或者预留一部分作为测试集
# 设实验次数为n
# for i in range(n):
#    将数据集分为训练集和测试集，每个类样本的在训练集和测试集比例为3：1
#    在训练集上合成数据训练模型
#    在测试集上测试结果
#取n次实验结果的平均值为最终成绩

# 我自己的算法不一样需要改一下

In [42]:
# 将少数类和多数类分开提取出来
major = []
major_label = []
minor = []
minor_label = []
for label_index in range(len(new_label)):
    if new_label[label_index] == 1.0:
        minor.append(scaled_data[label_index, :])
        minor_label.append(new_label[label_index])
    else:
        major.append(scaled_data[label_index, :])
        major_label.append(new_label[label_index])
# 定义gmean函数
def g_mean(ground_truth, prediction):
    matrix = confusion_matrix(y_true=ground_truth, y_pred=prediction)
    tpr = matrix[1, 1] / (matrix[1, 1] + matrix[1, 0])
    tpn = matrix[0, 0] / (matrix[0, 0] + matrix[0, 1])
    return np.sqrt(tpr * tpn)
result = dict([(key, {"gmean": list(), "fscore": list()}) for key in algorithm.keys()])
# 实验次数
n=100
for i in range(n):
    for key in algorithm.keys():
        alg = algorithm[key]
        alg.set_params(**alg_params[key])
        major_train, major_test, \
        major_label_train, major_label_test \
            = train_test_split(major, major_label, shuffle=True, test_size=0.25)
        minor_train, minor_test, \
        minor_label_train, minor_label_test \
            = train_test_split(minor, minor_label, shuffle=True, test_size=0.25)
        data_train = np.concatenate((major_train, minor_train), axis=0)
        label_train = np.concatenate((major_label_train, minor_label_train), axis=0)
        data_test = np.concatenate((major_test, minor_test), axis=0)
        label_test = np.concatenate((major_label_test, minor_label_test), axis=0)
        # smote生成合成样本
        kmsmote = KMeansSMOTE(kmeans_args={"n_clusters": 13})
        X_resampled, y_resampled = kmsmote.fit_sample(X=data_train, y=label_train)
        alg.fit(X=X_resampled, y=y_resampled)
        prediction = alg.predict(X=data_test)
        gmean = g_mean(label_test, prediction)
        fscore = f1_score(y_true=label_test, y_pred=prediction) / 2
        result[key]["gmean"].append(gmean)
        result[key]["fscore"].append(fscore)
new_line = 0       
for alg_key in result.keys():
    new_line += 1
    alg = result[alg_key]
    for score_key in alg.keys():
        mean_score = np.mean(alg[score_key])
        print(alg_key, " : ", score_key, " : ", mean_score, end="-----------------")
    print("\n")


knn  :  gmean  :  0.8913743085247064-----------------knn  :  fscore  :  0.4219403973314519-----------------

tree  :  gmean  :  0.8799849810771522-----------------tree  :  fscore  :  0.4160506474215546-----------------

svm  :  gmean  :  0.8388935498944136-----------------svm  :  fscore  :  0.39514476322256953-----------------

lgs  :  gmean  :  0.8491997265165628-----------------lgs  :  fscore  :  0.4000225866503977-----------------

nb  :  gmean  :  0.8500071309530675-----------------nb  :  fscore  :  0.4020581396531171-----------------



In [ ]:
# n_clusters = 1
# knn  :  gmean  :  0.9411238562522897-----------------knn  :  fscore  :  0.44569230006387967-----------------
# 
# tree  :  gmean  :  0.9045803402768919-----------------tree  :  fscore  :  0.4299026788304149-----------------
# 
# svm  :  gmean  :  0.8530002225541284-----------------svm  :  fscore  :  0.3917395822922544-----------------
# 
# lgs  :  gmean  :  0.8755260698096646-----------------lgs  :  fscore  :  0.4072211624073763-----------------
# 
# nb  :  gmean  :  0.884132349938559-----------------nb  :  fscore  :  0.4159771753974203-----------------

# 2
# knn  :  gmean  :  0.8972689514824813-----------------knn  :  fscore  :  0.41825619966268396-----------------
# 
# tree  :  gmean  :  0.9037606881517235-----------------tree  :  fscore  :  0.4319839869162522-----------------
# 
# svm  :  gmean  :  0.838893468714194-----------------svm  :  fscore  :  0.38320330340685804-----------------
# 
# lgs  :  gmean  :  0.8495532324119991-----------------lgs  :  fscore  :  0.39331976504622207-----------------
# 
# nb  :  gmean  :  0.8729411431510452-----------------nb  :  fscore  :  0.41663644607655104-----------------


#  3

# knn  :  gmean  :  0.8796484672970923-----------------knn  :  fscore  :  0.4096576379457141-----------------
# 
# tree  :  gmean  :  0.9031674212963534-----------------tree  :  fscore  :  0.4292420727186984-----------------
# 
# svm  :  gmean  :  0.8296867116133696-----------------svm  :  fscore  :  0.3790346629854126-----------------
# 
# lgs  :  gmean  :  0.8632652584749867-----------------lgs  :  fscore  :  0.40413380989969716-----------------
# 
# nb  :  gmean  :  0.8532775219545621-----------------nb  :  fscore  :  0.40275609135644735-----------------

# 4

# knn  :  gmean  :  0.8817672301515458-----------------knn  :  fscore  :  0.41251421437084373-----------------
# 
# tree  :  gmean  :  0.9001289923043703-----------------tree  :  fscore  :  0.4279798225124312-----------------
# 
# svm  :  gmean  :  0.8336894450133261-----------------svm  :  fscore  :  0.3841145718972338-----------------
# 
# lgs  :  gmean  :  0.8492269725114454-----------------lgs  :  fscore  :  0.3948871745889737-----------------
# 
# nb  :  gmean  :  0.8638401178512101-----------------nb  :  fscore  :  0.40996565986841277-----------------

# 5
# knn  :  gmean  :  0.8744954950136414-----------------knn  :  fscore  :  0.40569929110258945-----------------
# 
# tree  :  gmean  :  0.8948415457411769-----------------tree  :  fscore  :  0.42457282237680727-----------------
# 
# svm  :  gmean  :  0.838967820361812-----------------svm  :  fscore  :  0.3898658421378999-----------------
# 
# lgs  :  gmean  :  0.856675180114797-----------------lgs  :  fscore  :  0.4042799474509842-----------------
# 
# nb  :  gmean  :  0.8520927819676603-----------------nb  :  fscore  :  0.4070264420079164---------------

# # 6
# knn  :  gmean  :  0.8801319951405988-----------------knn  :  fscore  :  0.4131583525954028-----------------
# 
# tree  :  gmean  :  0.8844594518380531-----------------tree  :  fscore  :  0.4170189585932223-----------------
# 
# svm  :  gmean  :  0.8413456323886068-----------------svm  :  fscore  :  0.39403244493027095-----------------
# 
# lgs  :  gmean  :  0.85114734815387-----------------lgs  :  fscore  :  0.40166210663835744-----------------
# 
# nb  :  gmean  :  0.8545289531068694-----------------nb  :  fscore  :  0.4031196598277676-----------------

# 7
# knn  :  gmean  :  0.8823858665523493-----------------knn  :  fscore  :  0.4145628676597666-----------------
# 
# tree  :  gmean  :  0.8870629421828276-----------------tree  :  fscore  :  0.4190550963285638-----------------
# 
# svm  :  gmean  :  0.8419232032366147-----------------svm  :  fscore  :  0.39411325494053157-----------------
# 
# lgs  :  gmean  :  0.8471403462901916-----------------lgs  :  fscore  :  0.3981034689254329-----------------
# 
# nb  :  gmean  :  0.8493801969206392-----------------nb  :  fscore  :  0.40016166608518255-----------------

# 8
# knn  :  gmean  :  0.8844252912237122-----------------knn  :  fscore  :  0.413279426158549-----------------
# 
# tree  :  gmean  :  0.887384590217749-----------------tree  :  fscore  :  0.4196544228699352-----------------
# 
# svm  :  gmean  :  0.8385977471947318-----------------svm  :  fscore  :  0.39320747874689027-----------------
# 
# lgs  :  gmean  :  0.8671303246280947-----------------lgs  :  fscore  :  0.41197354975150097-----------------
# 
# nb  :  gmean  :  0.8544396598084643-----------------nb  :  fscore  :  0.4063565662541456-----------------


knn  :  gmean  :  0.8913743085247064-----------------knn  :  fscore  :  0.4219403973314519-----------------

tree  :  gmean  :  0.8799849810771522-----------------tree  :  fscore  :  0.4160506474215546-----------------

svm  :  gmean  :  0.8388935498944136-----------------svm  :  fscore  :  0.39514476322256953-----------------

lgs  :  gmean  :  0.8491997265165628-----------------lgs  :  fscore  :  0.4000225866503977-----------------

nb  :  gmean  :  0.8500071309530675-----------------nb  :  fscore  :  0.4020581396531171-----------------



In [ ]:
# # k 默认参数
# knn  :  gmean  :  0.8834169200011233-----------------knn  :  fscore  :  0.4153644275407602-----------------
# 
# tree  :  gmean  :  0.8984851325021005-----------------tree  :  fscore  :  0.4281771759524117-----------------
# 
# svm  :  gmean  :  0.8333796621367682-----------------svm  :  fscore  :  0.3911952545325008-----------------
# 
# lgs  :  gmean  :  0.8550369466867861-----------------lgs  :  fscore  :  0.40126563398931736-----------------
# 
# nb  :  gmean  :  0.8531845036797867-----------------nb  :  fscore  :  0.40145771424491433-----------------


In [11]:

# # smote_args={"k_neighbors": 1}
# knn  :  gmean  :  0.8747495707831638-----------------knn  :  fscore  :  0.4100305662945044-----------------
# 
# tree  :  gmean  :  0.8864548804343012-----------------tree  :  fscore  :  0.4203180542585993-----------------
# 
# svm  :  gmean  :  0.8375804018788833-----------------svm  :  fscore  :  0.39338634688883245-----------------
# 
# lgs  :  gmean  :  0.8466288754370366-----------------lgs  :  fscore  :  0.4006604779425519-----------------
# 
# nb  :  gmean  :  0.8471369424913686-----------------nb  :  fscore  :  0.3989790275617958-----------------

In [ ]:
# # # smote_args={"k_neighbors": 2}
# knn  :  gmean  :  0.8837774802207561-----------------knn  :  fscore  :  0.4151927964263234-----------------
# 
# tree  :  gmean  :  0.891563827361892-----------------tree  :  fscore  :  0.4212685479759082-----------------
# 
# svm  :  gmean  :  0.8362142721799632-----------------svm  :  fscore  :  0.3908543439707608-----------------
# 
# lgs  :  gmean  :  0.8461557443434506-----------------lgs  :  fscore  :  0.39704443878109735-----------------
# 
# nb  :  gmean  :  0.8564571631592243-----------------nb  :  fscore  :  0.40489333139593214------------
# 

# # smote_args={"k_neighbors": 3}
# knn  :  gmean  :  0.8809110595094961-----------------knn  :  fscore  :  0.4160085086133812-----------------
# 
# tree  :  gmean  :  0.8827042101675233-----------------tree  :  fscore  :  0.420011979078171-----------------
# 
# svm  :  gmean  :  0.8461309525215281-----------------svm  :  fscore  :  0.398165633674566-----------------
# 
# lgs  :  gmean  :  0.8519837058402604-----------------lgs  :  fscore  :  0.39847981924394843-----------------
# 
# nb  :  gmean  :  0.860249158435178-----------------nb  :  fscore  :  0.4085970618170767--------------

# # smote_args={"k_neighbors": 4}

# knn  :  gmean  :  0.8894811178733582-----------------knn  :  fscore  :  0.42004545930864795-----------------
# 
# tree  :  gmean  :  0.8905366180517889-----------------tree  :  fscore  :  0.4203758980234164-----------------
# 
# svm  :  gmean  :  0.8360681163242991-----------------svm  :  fscore  :  0.3944780173084205-----------------
# 
# lgs  :  gmean  :  0.8565785990745606-----------------lgs  :  fscore  :  0.4057283784030499-----------------
# 
# nb  :  gmean  :  0.8589726005116437-----------------nb  :  fscore  :  0.40689868736808094-----------------


# # smote_args={"k_neighbors": 5}
# knn  :  gmean  :  0.8804277670550549-----------------knn  :  fscore  :  0.41039431530360887-----------------
# 
# tree  :  gmean  :  0.8888285045112433-----------------tree  :  fscore  :  0.4210453143666351-----------------
# 
# svm  :  gmean  :  0.834428415726699-----------------svm  :  fscore  :  0.38978965641819235-----------------
# 
# lgs  :  gmean  :  0.8599363757187781-----------------lgs  :  fscore  :  0.40465802348575963-----------------
# 
# nb  :  gmean  :  0.8677092959299015-----------------nb  :  fscore  :  0.41124180775295066-----------------


# smote_args={"k_neighbors": 6}
# knn  :  gmean  :  0.8792316524738334-----------------knn  :  fscore  :  0.4112754944797625-----------------
# 
# tree  :  gmean  :  0.8943210070282658-----------------tree  :  fscore  :  0.42580981470592844-----------------
# 
# svm  :  gmean  :  0.8383433769158539-----------------svm  :  fscore  :  0.39149495230900655-----------------
# 
# lgs  :  gmean  :  0.8614857234966116-----------------lgs  :  fscore  :  0.40943239620615934-----------------
# 
# nb  :  gmean  :  0.8600734315426357-----------------nb  :  fscore  :  0.4076159240062844-----------------

# smote_args={"k_neighbors": 6}

# knn  :  gmean  :  0.8918497068478243-----------------knn  :  fscore  :  0.4214347561264789-----------------
# 
# tree  :  gmean  :  0.8914440794737114-----------------tree  :  fscore  :  0.42256507728403975-----------------
# 
# svm  :  gmean  :  0.834087741073317-----------------svm  :  fscore  :  0.3923956997933956-----------------
# 
# lgs  :  gmean  :  0.8508457286768089-----------------lgs  :  fscore  :  0.4043254873155125-----------------
# 
# nb  :  gmean  :  0.8664179919262414-----------------nb  :  fscore  :  0.40853693825981374-----------------


# smote_args={"k_neighbors": 7}
# knn  :  gmean  :  0.9002557906833166-----------------knn  :  fscore  :  0.4243463898351079-----------------
# 
# tree  :  gmean  :  0.886910038426569-----------------tree  :  fscore  :  0.418690068158709-----------------
# 
# svm  :  gmean  :  0.8346121458238791-----------------svm  :  fscore  :  0.39165784054082403-----------------
# 
# lgs  :  gmean  :  0.8454637179100963-----------------lgs  :  fscore  :  0.39616046048806547-----------------
# 
# nb  :  gmean  :  0.8591389716995066-----------------nb  :  fscore  :  0.40442289811550053-----------------


# smote_args = {"k_neighbors": 8}
# knn  :  gmean  :  0.8927896901380177-----------------knn  :  fscore  :  0.4194088842038399-----------------
# 
# tree  :  gmean  :  0.8824635787716448-----------------tree  :  fscore  :  0.41517810202395244-----------------
# 
# svm  :  gmean  :  0.8416840732711445-----------------svm  :  fscore  :  0.39596163452168825-----------------
# 
# lgs  :  gmean  :  0.8611351781438471-----------------lgs  :  fscore  :  0.40402978733555467-----------------
# 
# nb  :  gmean  :  0.8597261161292689-----------------nb  :  fscore  :  0.40457261879938045-----------------


# smote_args = {"k_neighbors": 9}
# knn  :  gmean  :  0.8856756180442085-----------------knn  :  fscore  :  0.41503866794541466-----------------
# 
# tree  :  gmean  :  0.890178075172206-----------------tree  :  fscore  :  0.42007517695808544-----------------
# 
# svm  :  gmean  :  0.83225409548464-----------------svm  :  fscore  :  0.3900966087266174-----------------
# 
# lgs  :  gmean  :  0.8565669461502449-----------------lgs  :  fscore  :  0.4049773185834655-----------------
# 
# nb  :  gmean  :  0.849312510566516-----------------nb  :  fscore  :  0.4024032811942581-----------------

#  smote_args = {"k_neighbors": 10}

# knn  :  gmean  :  0.8786603137348699-----------------knn  :  fscore  :  0.4133779704760891-----------------
# 
# tree  :  gmean  :  0.8924007324583171-----------------tree  :  fscore  :  0.4224888730903911-----------------
# 
# svm  :  gmean  :  0.8166778611213542-----------------svm  :  fscore  :  0.3805642209725418-----------------
# 
# lgs  :  gmean  :  0.8576408453622807-----------------lgs  :  fscore  :  0.4047546914297906-----------------
# 
# nb  :  gmean  :  0.8478664183023656-----------------nb  :  fscore  :  0.4012033970471752-----------------

